# Object Detection Using API

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageColor, ImageDraw, ImageFont, Image, ImageOps
import time
import tempfile

# Six is a Python 2 and 3 compatibility library. 
# It provides utility functions for smoothing over the differences between the 
# Python versions with the goal of writing Python code that is compatible on 
# both Python versions.
from six.moves.urllib.request import urlopen
from six import BytesIO

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

### Download the model from Tensorflow Hub

Tensorflow Hub is a repository of trained machine learning models which you can reuse in your own projects. 
- You can see the domains covered [here](https://tfhub.dev/) and its subcategories. 
- For this lab, you will want to look at the [image object detection subcategory](https://tfhub.dev/s?module-type=image-object-detection). 
- You can select a model to see more information about it and copy the URL so you can download it to your workspace. 


[inception resnet version 2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1)

[ssd mobilenet version 2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2)

In [ ]:
# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# ssd mobilenet version 2
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

# load the model
model = hub.load(module_handle)

# take a look at the available signatures for this particular model
# different signature can be used for different tasks
print(model.signatures.keys())

# choose the default signature
# For object detection models, its 'default' signature will accept a batch of 
# image tensors and output a dictionary describing the objects detected
detector = model.signatures['default']

### download_and_resize_image

This function downloads an image specified by a given "url", pre-processes it, and then saves it to disk.

In [ ]:
def download_and_resize_image(url, new_width=256, new_height=256, display = False):
    '''
    Fetches an image online, resizes it and saves it locally.
    
    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image
        
    Returns:
        (string) -- path to the saved image
    '''
    
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    # the Image.Antialias renamed to Image.Lanczos
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)

    if (display):
      fig = plt.figure(figsize = (20, 15))
      plt.grid(False)
      plt.imshow(pil_image_rgb)
    
    return filename

### Download and preprocess an image

Now, using `download_and_resize_image` you can get a sample image online and save it locally. 
- We've provided a URL for you, but feel free to choose another image to run through the object detector.
- You can use the original width and height of the image but feel free to modify it and see what results you get.

In [ ]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/b/b3/I-90-94_Entrance_at_Madison_Street%2C_Chicago_%2814560285196%29.jpg"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592, True)

## Prepare for Visualization


In [ ]:
def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color, font, thickness=6, display_str_list=()):

    """
    Adds a bounding box to an image.
    
    Args:
        image -- the image object
        ymin -- bounding box coordinate
        xmin -- bounding box coordinate
        ymax -- bounding box coordinate
        xmax -- bounding box coordinate
        color -- color for the bounding box edges
        font -- font for class label
        thickness -- edge thickness of the bounding box
        display_str_list -- class labels for each object detected
    
    
    Returns:
        No return.  The function modifies the `image` argument 
                    that gets passed into this function
    
    """
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    
    # scale the bounding box coordinates to the height and width of the image
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    
    # define the four edges of the detection box
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
        
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """
    Overlay labeled boxes on an image with formatted scores and label names.
    
    Args:
        image -- the image as a numpy array
        boxes -- list of detection boxes
        class_names -- list of classes for each detected object
        scores -- numbers showing the model's confidence in detecting that object
        max_boxes -- maximum detection boxes to overlay on the image (default is 10)
        min_score -- minimum score required to display a bounding box
    
    Returns:
        image -- the image after detection boxes and classes are overlaid on the original image.
    """
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              35)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
        
        # only display detection boxes that have the minimum score or higher
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                         int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")

            # draw one bounding box and overlay the class labels onto the image
            draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, font, display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
        
    return image

### run_detector

This function will take in the object detection model `detector` and the path to a sample image, then use this model to detect objects and display its predicted class categories and detection boxes.
- run_detector uses `load_image` to convert the image into a tensor.

In [ ]:
# load an image and turn to a tensor
img = tf.io.read_file(downloaded_image_path)
img = tf.image.decode_jpeg(img, channels = 3)

# add a batch dimension in front of the tensor
# converted_img.shape = [1, 2592, 3872, 3]
converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

start_time = time.time()
result = detector(converted_img)
end_time = time.time()

# save the results in a dictionary
result = {key:value.numpy() for key,value in result.items()}

# print results
print("Found %d objects." % len(result["detection_scores"]))
print("Inference Time: ", end_time - start_time)

# print(result["detection_scores"]) # detection scores
# print(result["detection_class_entities"]) # classes
# print(result["detection_boxes"]) # bounding box

image_with_boxes = draw_boxes(img.numpy(), result['detection_boxes'], result['detection_class_entities'], result['detection_scores'])

# display image

fig = plt.figure(figsize = (20, 15))
plt.grid(False)
plt.imshow(image_with_boxes)
